In [1]:
import pandas as pd

from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [2]:
# NOTE: Custome Module !!
from module.dataframe import df_query_by_type, series_string_filter, pred_muti_model
from module.preprocess import do_preprocessing
from module.util import save_pkl,now

In [3]:
SKLEARN_COMMON_PARAMSET = {"random_state":42}
STARTED_AT = now()

---

In [4]:
train = pd.read_parquet("./database/train.parquet")
test = pd.read_parquet("./database/test.parquet")

In [5]:
# least 2min
train, test = do_preprocessing(train, test)

In [6]:
__train_x, __train_y = train.drop(columns="target"), train["target"]
__test_x = test

x_train, x_valid, y_train, y_valid = train_test_split(__train_x, __train_y, test_size=0.05, **SKLEARN_COMMON_PARAMSET)

---

In [7]:
import numpy as np
import lightgbm as lgb

from sklearn.metrics import r2_score as R2
from sklearn.metrics import median_absolute_error as MAE

from sklearn.ensemble import RandomForestRegressor,BaggingRegressor
from sklearn.linear_model import LinearRegression

In [8]:
REG_RF_PARAMS = {"n_estimators": 40, "max_depth": 15, "min_samples_split": 4, **SKLEARN_COMMON_PARAMSET}
reg_rforest = RandomForestRegressor(**REG_RF_PARAMS)

BAGGING_ENC_PARAMS = {
    "estimator": reg_rforest,
    "n_estimators": 15,
    "max_samples": 0.8,
    **SKLEARN_COMMON_PARAMSET,
}
bagging_ens_model = BaggingRegressor(**BAGGING_ENC_PARAMS, n_jobs=-1)

In [9]:
# 24M
bagging_ens_model.fit(x_train, y_train)

BaggingRegressor(estimator=RandomForestRegressor(max_depth=15,
                                                 min_samples_split=4,
                                                 n_estimators=40,
                                                 random_state=42),
                 max_samples=0.8, n_estimators=15, n_jobs=-1, random_state=42)

In [10]:
final_model = bagging_ens_model
[MAE(final_model.predict(x_valid), y_valid), R2(final_model.predict(x_valid), y_valid)]

[5.798376907243728, 0.4211265003898873]

## Submission
---

In [11]:
sample_submission = pd.read_csv('./database/sample_submission.csv')

In [12]:
save_pkl(f"./model/begging-{STARTED_AT}.pkl", final_model)
sample_submission["target"] = final_model.predict(__test_x)
sample_submission.to_csv(f"./submission-file-{STARTED_AT}.csv", index=False)
sample_submission

,id,target
0,TEST_000000,25.058459
1,TEST_000001,46.584368
2,TEST_000002,48.946978
3,TEST_000003,37.227816
4,TEST_000004,40.226237
...,...,...
291236,TEST_291236,48.073143
291237,TEST_291237,38.473714
291238,TEST_291238,23.126805
291239,TEST_291239,25.852805


![decisiontree](./0004.png)
![linear](./0004.png)